In [ ]:
import util, perceptron, nb, samples
from sklearn.svm import SVC
import numpy as np
import random
!pip3 install dill
import dill
import time

In [ ]:
DIGIT_DATUM_WIDTH=28
DIGIT_DATUM_HEIGHT=28
FACE_DATUM_WIDTH=60
FACE_DATUM_HEIGHT=70

DATUM_WIDTH = DIGIT_DATUM_WIDTH
DATUM_HEIGHT = DIGIT_DATUM_HEIGHT

ITERATIONS = 10
DATASET = 'digits' # 'digits' or 'faces'
TRAIN_PERCENT = 100
TEST_PERCENT = 100
CLASSIFIER = 'NaiveBayes' # 'Perceptron' or 'NaiveBayes'
UseNewFeatureExtractor = False

classifiers = {'Perceptron': perceptron.Perceptron,
               'NaiveBayes': nb.NaiveBayesClassifier}




numTraining = 5000 if DATASET == 'digits' else 451
numTest = 1000 if DATASET == 'digits' else 150
# numTraining = int(numTraining * TRAIN_PERCENT / 100)
# numTest = int(numTest * TEST_PERCENT / 100)
# getFeatures = features[DATASET]

In [ ]:
def modifyFeatures(datapoint):
    global DATUM_WIDTH, DATUM_HEIGHT
    data = ([[None for i in range(DATUM_WIDTH)] for j in range(DATUM_HEIGHT)])
    for i in range(len(data)):
        for j in range(len(data[i])):
            data[i][j] = datapoint[(i,j)]
    return np.array(data)
  
  
def basicFeatureExtractorDigit(datum):
  """
  Returns a set of pixel features indicating whether
  each pixel in the provided datum is white (0) or gray/black (1)
  """
  a = datum.getPixels()

  features = util.Counter()
  for x in range(DIGIT_DATUM_WIDTH):
    for y in range(DIGIT_DATUM_HEIGHT):
      if datum.getPixel(x, y) > 0:
        features[(x,y)] = 1
      else:
        features[(x,y)] = 0
  return features

def fourSquareFeatureExtractorDigit(datum):
  features = util.Counter()
  for x in range(0,DIGIT_DATUM_WIDTH,4):
    for y in range(0,DIGIT_DATUM_HEIGHT,4):
      templi = []
      for k in range(4):
        flag = False
        for l in range(4):
          if datum.getPixel(x+k,y+l) > 0:
            features[(x,y)] = 1
            flag = True
            break
          else:
            features[(x,y)] = 0

        if flag == True:
          break
  return features

def basicFeatureExtractorFace(datum):
  """
  Returns a set of pixel features indicating whether
  each pixel in the provided datum is an edge (1) or no edge (0)
  """
  a = datum.getPixels()

  features = util.Counter()
  for x in range(FACE_DATUM_WIDTH):
    for y in range(FACE_DATUM_HEIGHT):
      if datum.getPixel(x, y) > 0:
        features[(x,y)] = 1
      else:
        features[(x,y)] = 0
  return features

def fiveSquareFeatureExtractorFace(datum):
  features = util.Counter()
  for x in range(0,FACE_DATUM_WIDTH,5):
    for y in range(0,FACE_DATUM_HEIGHT,5):
      templi = []
      for k in range(5):
        flag = False
        for l in range(5):
          if datum.getPixel(x+k,y+l) > 0:
            features[(x,y)] = 1
            flag = True
            break
          else:
            features[(x,y)] = 0

        if flag == True:
          break
  return features

features = {'digits': basicFeatureExtractorDigit, 'faces': basicFeatureExtractorFace}
NewFeatures = {'digits': fourSquareFeatureExtractorDigit, 'faces': fiveSquareFeatureExtractorFace}

In [ ]:
def prepData(dataset):
    numTraining = 5000 if dataset == 'digits' else 451
    numTest = 1000 if dataset == 'digits' else 150
    global DATUM_WIDTH, DATUM_HEIGHT
    # numTest = int(numTest * TEST_PERCENT / 100)
    if UseNewFeatureExtractor:
        getFeatures = NewFeatures[dataset]
    else:
        getFeatures = features[dataset]
    legalLabels = list(range(10)) if dataset == 'digits' else list(range(2))
    DATUM_WIDTH = DIGIT_DATUM_WIDTH if dataset == 'digits' else FACE_DATUM_WIDTH
    DATUM_HEIGHT = DIGIT_DATUM_HEIGHT if dataset == 'digits' else FACE_DATUM_HEIGHT
    legalLabels = list(range(10)) if dataset == 'digits' else list(range(2))
    if(dataset=="faces"):
        rawTrainingData = samples.loadDataFile("data/facedata/facedatatrain", numTraining,DATUM_WIDTH,DATUM_HEIGHT)
        trainingLabels = samples.loadLabelsFile("data/facedata/facedatatrainlabels", numTraining)
        rawValidationData = samples.loadDataFile("data/facedata/facedatatrain", numTest,DATUM_WIDTH,DATUM_HEIGHT)
        validationLabels = samples.loadLabelsFile("data/facedata/facedatatrainlabels", numTest)
        rawTestData = samples.loadDataFile("data/facedata/facedatatest", numTest,DATUM_WIDTH,DATUM_HEIGHT)
        testLabels = samples.loadLabelsFile("data/facedata/facedatatestlabels", numTest)
    else:
        rawTrainingData = samples.loadDataFile("data/digitdata/trainingimages", numTraining,DATUM_WIDTH,DATUM_HEIGHT)
        trainingLabels = samples.loadLabelsFile("data/digitdata/traininglabels", numTraining)
        rawValidationData = samples.loadDataFile("data/digitdata/validationimages", numTest,DATUM_WIDTH,DATUM_HEIGHT)
        validationLabels = samples.loadLabelsFile("data/digitdata/validationlabels", numTest)
        rawTestData = samples.loadDataFile("data/digitdata/testimages", numTest,DATUM_WIDTH,DATUM_HEIGHT)
        testLabels = samples.loadLabelsFile("data/digitdata/testlabels", numTest)
        
    return rawTrainingData, trainingLabels, rawValidationData, validationLabels, rawTestData, testLabels, legalLabels, numTraining, getFeatures


def sample_data(rawTrainingData, trainingLabels, train_percent, numTraining):
    
    
    # print(len(rawTrainingData))
    finalTrainData = []
    for i in range(len(rawTrainingData)):
        # print(i)
        finalTrainData.append((rawTrainingData[i], trainingLabels[i]))
        
    numTraining = int(numTraining * train_percent / 100)
    
    finalTrainData = random.sample(finalTrainData, numTraining)
        
    # finalValidationData = []
    # for i in range(len(rawValidationData)):
    #     finalValidationData.append((rawValidationData[i], validationLabels[i]))
        
    # finalTestData = []
    # for i in range(len(rawTestData)):
    #     finalTestData.append((rawTestData[i], testLabels[i]))
    
    rawTrainingData = []
    trainingLabels = []
    for i in range(len(finalTrainData)):
        rawTrainingData.append(finalTrainData[i][0])
        trainingLabels.append(finalTrainData[i][1])
        
    return rawTrainingData, trainingLabels#, rawValidationData, validationLabels, rawTestData, testLabels


def getDataForClassification(rawTrainingData, trainingLabels, rawValidationData, validationLabels, rawTestData, testLabels, legalLabels, train_percent, numTraining, getFeatures):

    sampled_raw_training_data, trainingLabels = sample_data(rawTrainingData, trainingLabels, train_percent, numTraining)
    
    trainingData = list(map(getFeatures, sampled_raw_training_data))
    validationData = list(map(getFeatures, rawValidationData))
    testData = list(map(getFeatures, rawTestData))
    
    return trainingData, trainingLabels, validationData, validationLabels, testData, testLabels, legalLabels
    
    
    

In [ ]:
def runClassifiers(trainingData, trainingLabels, validationData, validationLabels, testData, testLabels, legalLabels):
    results = {}
    start = time.time()
    model = perceptron.Perceptron(legalLabels, max_iterations=10)
    model.train(trainingData, trainingLabels, validationData, validationLabels)
    guesses = model.classify(testData)
    correct = [guesses[i] == testLabels[i] for i in range(len(testLabels))].count(True)
    accuracy = 100 * correct / len(testLabels)
    print("Perceptron", time.time() - start)
    results['Perceptron'] = accuracy
    start = time.time()
    model = nb.NaiveBayesClassifier(legalLabels)
    model.train(trainingData, trainingLabels, validationData, validationLabels)
    guesses = model.classify(testData)
    correct = [guesses[i] == testLabels[i] for i in range(len(testLabels))].count(True)
    accuracy = 100 * correct / len(testLabels)
    print("NB", time.time() - start)
    results['NaiveBayes'] = accuracy
    
    newTrainingData = np.array(list(map(modifyFeatures, trainingData)))
    nsamples, nx, ny = newTrainingData.shape
    d2_train_dataset = newTrainingData.reshape((nsamples,nx*ny))
    
    newTestData = np.array(list(map(modifyFeatures, testData)))
    nsamples, nx, ny = newTestData.shape
    d2_test_dataset = newTestData.reshape((nsamples,nx*ny))
    start = time.time()
    model = SVC().fit(d2_train_dataset, trainingLabels)
    guesses = list(model.predict(d2_test_dataset))
    correct = [guesses[i] == testLabels[i] for i in range(len(testLabels))].count(True)
    accuracy = 100 * correct / len(testLabels)
    
    print("SVC", time.time() - start)
    results['SVC'] = accuracy
    
    return results
    

In [ ]:
train_percents = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
final_results = {x:{x:[] for x in ['Perceptron', 'NaiveBayes', 'SVC']} for x in train_percents}
xrawTrainData, xtrainingLabels, rawValidationData, validationLabels, rawTestData, testLabels, legalLabels, numTraining, getFeatures = prepData(dataset='faces')

In [ ]:
len(xrawTrainData)

In [ ]:
final_results

In [ ]:
for percent in train_percents:
    print("Training Data Size: ",(percent))
    for _ in range(1):
        trainingData, trainingLabels, validationData, validationLabels, testData, testLabels, legalLabels = getDataForClassification(xrawTrainData, xtrainingLabels, rawValidationData, validationLabels, rawTestData, testLabels, legalLabels, percent, numTraining, getFeatures)
        
        results = runClassifiers(trainingData, trainingLabels, validationData, validationLabels, testData, testLabels, legalLabels)
        
        for res in results.keys():
            final_results[percent][res].append(results[res])
        
        # final_results[percent].append(results)
    # print(final_results)
    # dill.dump(final_results, open('results_faces.p', 'wb'))
    

In [ ]:
dill.dump(final_results, open('results.p', 'wb'))